In [54]:
from upd_mn import *
import gzip
from io import StringIO, BytesIO
import shutil
import pandas as pd
import os
import csv
import teradatasql

In [110]:
cols_dict = {'CDRDate1': 'CDR_DATE',
             'START_DATE': 'CDR_DATE',
             'Carrier1': 'CARRIER',
             'Traffic_Direction21': 'DIRECTION',
             'Destination1': 'DESTINATION',
             'TotalDurationInMinutes_Sum': 'MINUTES',
             'DURATION': 'MINUTES',
             'Total_Calls_Connected': 'TOTAL_CALLS_CONNECTED',
             'Total_Cost': 'COST',
             'Rate_List_Currency1': 'CURRENCY',
             'CDR_Date': 'CDR_DATE',
             'GlobalCarrier': 'CARRIER',
             'Branch': 'BRANCH',
             'Destination_Band': 'DESTINATION',
             'Minutes': 'MINUTES',
             'Cost': 'COST',
             'CDRDate_Value': 'CDR_DATE',
             'GlobalCarrier_Value': 'CARRIER',
             'Region_Value': 'BRANCH',
             'Destination_Value': 'DESTINATION',
             'TotalDurationInMinutes_Value': 'MINUTES',
             'CARRIER_SHORT_NAME': 'CARRIER'
             }

In [3]:
source='product_reporting@tele2.ru'

In [244]:
account = log_into_account(mailbox=source)  # 'igor.i.plotnikov@tele2.ru'
last_msgs = get_messages(account, folder='МН_ИТК', number_of_days=1)

Logging in...

Checking email...



In [248]:
import tarfile

In [259]:
import os.path

In [265]:
os.path.abspath('')

'C:\\Users\\igor.i.plotnikov\\PycharmProjects\\work\\ИК'

In [264]:
os.path

<module 'ntpath' from 'c:\\users\\igor.i.plotnikov\\appdata\\local\\programs\\python\\python38\\lib\\ntpath.py'>

In [268]:
cur_dir = os.path.abspath('') + '\\tmp'
cur_dir

'C:\\Users\\igor.i.plotnikov\\PycharmProjects\\work\\ИК\\tmp'

In [273]:
dt = '06/04/2021 13:04:56'

In [280]:
new_dt = None
for date_format in ('%d.%m.%Y', '%d.%m.%y', '%d/%m/%Y', '%d.%m.%Y %H:%M:%S', '%d.%m.%y %H:%M:%S', '%d/%m/%Y %H:%M:%S'):
    try:
        new_dt = datetime.strptime(dt, date_format).strftime('%Y-%m-%d')
    except ValueError:
        continue
    else:
        break
print(new_dt)

2021-04-06


In [292]:
for msg in last_msgs:
    for attachment in msg.attachments:
        if isinstance(attachment, FileAttachment) and attachment.name.endswith('.txt.gz'):
            filepath = os.path.join(os.path.abspath('')+'\\tmp', attachment.name)
            with open(filepath, 'wb') as f:
                f.write(attachment.content)
            break
df = pd.read_csv(filepath, compression='gzip', header=0, quoting=3, sep=';',engine='python')
for col in df.columns:
    df[col] = df[col].apply(str).str.replace('"', '')
df.head()

,#GLOBALCARRIER,CARRIER_SHORT_NAME,MACROREGION,OPERATOR_NAME,START_DATE,COST,DURATION
0,T_GTK,T_GTK T2 SPB,СНГ. Украина,Украина (мобил.)-UMC,03.06.2021,4.400000,.550000
1,M_MTS,M_MTS T2 NSK,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,05.06.2021,21.794000,21.366667
2,M_MGF,M_MGF T2 SPB,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,02.06.2021,.969000,.950000
3,T_INT,T_INT DZ BRT,ЕС. Великобритания,Великобритания (мобил.) - Vodafone,06.06.2021,.047500,.050000
4,M_MTS,M_MTS RM EKT,ЕС. Балтия. Эстония,Эстония,05.06.2021,.063333,.066667


In [271]:
filepath

'C:\\Users\\igor.i.plotnikov\\PycharmProjects\\work\\ИК\\tmp\\20210601-07.txt.gz'

In [287]:
df = pd.read_csv(filepath, compression='gzip', header=0, quoting=csv.QUOTE_NONNUMERIC, sep=';',engine='python')
df.head()

ValueError: could not convert string to float: '#GLOBALCARRIER'

In [286]:
df.apply(lambda x: x.str.replace('"', ''))

,#GLOBALCARRIER,CARRIER_SHORT_NAME,MACROREGION,OPERATOR_NAME,START_DATE,COST,DURATION
0,T_GTK,T_GTK T2 SPB,СНГ. Украина,Украина (мобил.)-UMC,03.06.2021,4.400000,.550000
1,M_MTS,M_MTS T2 NSK,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,05.06.2021,21.794000,21.366667
2,M_MGF,M_MGF T2 SPB,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,02.06.2021,.969000,.950000
3,T_INT,T_INT DZ BRT,ЕС. Великобритания,Великобритания (мобил.) - Vodafone,06.06.2021,.047500,.050000
4,M_MTS,M_MTS RM EKT,ЕС. Балтия. Эстония,Эстония,05.06.2021,.063333,.066667
...,...,...,...,...,...,...,...
65642,T_GTK,T_GTK SK MOS,СНГ. Армения ДК Билайн,Армения (мобил.) - Armentel,05.06.2021,18.291667,4.783333
65643,T_INT,T_INT RM HAN,Азия. Катар,Катар (мобил.),06.06.2021,1.662500,1.750000
65644,T_VPM,T_VPM T2 RYZ,СНГ. Туркменистан,Туркменистан,01.06.2021,187.750000,12.516667
65645,T_VPM,T_VPM DZ SRV,ЕС. Болгария,Болгария (мобил.) - Mobitell,01.06.2021,313.000000,20.866667


In [282]:
for msg in last_msgs:
    for attachment in msg.attachments:
        if isinstance(attachment, FileAttachment) and attachment.name.endswith('.txt.gz'):
            att_name = attachment.name
        break
df = pd.read_csv(att_name, compression='gzip', header=0, quoting=2, doublequote=True, sep=';',engine='python')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '20210601-07.txt.gz'

In [232]:
att_name

'ITC_CALLS_MONTH_20210501.txt.gz'

In [109]:
for msg in last_msgs:
    for attachment in msg.attachments:
        if isinstance(attachment, FileAttachment) and attachment.name.endswith('.txt.gz'):
            df = pd.read_csv(attachment.name, compression='gzip', header=0, quoting=3, sep=';',engine='python')
            df = df.apply(lambda x: x.str.replace('"', ''))
            df['DURATION'] = df['DURATION'].apply(lambda x: pd.to_numeric(x))
            df['COST'] = df['COST'].apply(lambda x: pd.to_numeric(x))
            df.rename(cols_dict, axis=1, inplace=True)
            df.drop([col for col in ['#GLOBALCARRIER','MACROREGION'] if col in df.columns], axis=1, inplace=True)
            df[['CARRIER', 'BRANCH']] = df['CARRIER'].str.extract(r'([A-Z]{1}\_[A-Z0-9]{3})\s([[A-Z0-9]{2}\s[A-Z0-9]*)', expand=True, )
            df['CARRIER_TYPE'] = df['CARRIER'].str.extract(r'([A-Z]{1})\_[A-Z0-9]{3}', expand=True, )
            df = df[(df['CARRIER_TYPE'].isin(['T','H'])) & (~df['CARRIER'].isin(['T_INT','H_RTK']))]
            break
            
#             f = gzip.open(attachment.name, 'r')
#             print(f)
#             print()
#             msg_datetime_str = msg.datetime_received.replace(tzinfo=None).strftime("%Y-%m-%d %H-%M-%S")
#             print(f'Importing {attachment.name} received at {msg_datetime_str}...')
#             txt_report=f.read()
#             print(txt_report)

In [115]:
df_ok = df.copy()
df_ok.head()

c:\users\igor.i.plotnikov\appdata\local\programs\python\python38\lib\site-packages\pandas\core\strings.py:908: FutureWarning: Possible nested set at position 27
  regex = re.compile(pat, flags=flags)


,CARRIER,OPERATOR_NAME,CDR_DATE,COST,MINUTES,BRANCH
0,T_VPM,Болгария (мобил.) - BTC Mobile,16.05.2021,3.250000,0.216667,DZ BRT
1,H_CHM,ОАЭ (мобил.) - Mobile others,15.05.2021,9710.054557,673.683333,T2 CFN
2,T_INT,Беларусь (мобил.) - MTS,14.05.2021,8.518333,8.966667,T2 KIR
3,T_INT,Венгрия Budapest,12.05.2021,5.383333,5.666667,T2 VRN
4,M_MTS,Казахстан (мобил.) - Mobile others,02.05.2021,6.613000,6.483333,T2 VRN


In [126]:
# затирать во входящем
# T_INT
# H_RTK 
# не с Т
# не с H

# затирать в исходящем
# T_INT 
# H_RTK

df_ok = df_ok[(df_ok['CARRIER_TYPE'].isin(['T','H'])) & (~df_ok['CARRIER'].isin(['T_INT','H_RTK']))]
df.reset_index(inplace=True, drop=True)
df_ok.head()

,CARRIER,OPERATOR_NAME,CDR_DATE,COST,MINUTES,BRANCH,CARRIER_TYPE
0,T_VPM,Болгария (мобил.) - BTC Mobile,16.05.2021,3.250000,0.216667,DZ BRT,T
1,H_CHM,ОАЭ (мобил.) - Mobile others,15.05.2021,9710.054557,673.683333,T2 CFN,H
6,T_VPM,Беларусь (мобил.) - BTN BeST,05.05.2021,44.500000,2.966667,RM PRM,T
8,T_VPM,Швеция (мобил.) - Comviq,18.05.2021,1.500000,0.100000,T2 TUL,T
10,T_VPM,Эстония (мобил.) - Tele2,15.05.2021,1072.500000,71.500000,T2 TAM,T


In [131]:
query = '''
    sel * from uat_product.incoming_mn_dic
'''

with teradatasql.connect() as session:
    mn_dic = pd.read_sql(query, session)
mn_dic.columns = ['OPERATOR_NAME','COUNTRY','AREA']

df_new['DIRECTION'] = 'Incoming'
df_new['MINUTES'] = df_new['MINUTES'].apply(lambda x: pd.to_numeric(x, downcast='float'))
df_new['INSERT_DATE'] = datetime.today().strftime('%Y-%m-%d')
df_new = pd.merge(df_ok, mn_dic, how='left', left_on='OPERATOR_NAME', right_on='OPERATOR_NAME')
df_new.head()

,CARRIER,OPERATOR_NAME,CDR_DATE,COST,MINUTES,BRANCH,CARRIER_TYPE,ORIGINATSIJA,PRINADLEZHNOSTREGION
0,T_VPM,Болгария (мобил.) - BTC Mobile,16.05.2021,3.250000,0.216667,DZ BRT,T,Болгария,Европа
1,H_CHM,ОАЭ (мобил.) - Mobile others,15.05.2021,9710.054557,673.683333,T2 CFN,H,ОАЭ,Азия
2,T_VPM,Беларусь (мобил.) - BTN BeST,05.05.2021,44.500000,2.966667,RM PRM,T,Беларусь,СНГ
3,T_VPM,Швеция (мобил.) - Comviq,18.05.2021,1.500000,0.100000,T2 TUL,T,Швеция,Европа
4,T_VPM,Эстония (мобил.) - Tele2,15.05.2021,1072.500000,71.500000,T2 TAM,T,Эстония,Европа


In [1]:
import pandas as pd

In [6]:
df = pd.read_csv(r"C:\Users\igor.i.plotnikov\PycharmProjects\work\MN\Отчеты\2021-09-13 от Насти за фев-май\Outgoing MN direct daily за весь прошлый месяц Апрель.csv", 
    sep=';', thousands=',', engine='python')
df.head()

ParserError: Expected 5 fields in line 1925, saw 6

In [174]:
for msg in last_msgs:
    for attachment in msg.attachments:
        if isinstance(attachment, FileAttachment) and attachment.name.endswith('.csv') \
        and attachment.name.startswith('Outgoing MN direct daily'):
            msg_datetime_str = msg.datetime_received.replace(tzinfo=None).strftime("%Y-%m-%d %H-%M-%S")
            print(f'Importing {attachment.name} received at {msg_datetime_str}...')
            csv_report = io.StringIO(attachment.content.decode('utf-8'))
            raw = pd.read_csv(csv_report, thousands=',')
            break
raw.head()

Importing Outgoing MN direct daily.csv received at 2021-06-09 07-52-36...
Importing Outgoing MN direct daily.csv received at 2021-06-08 07-52-24...
Importing Outgoing MN direct daily.csv received at 2021-06-07 07-53-25...
Importing Outgoing MN direct daily.csv received at 2021-06-06 07-52-22...


,CDR_Date,Branch,GlobalCarrier,Destination_Band,Minutes,Cost,Tariff,Rate_List_Currency1
0,01.06.2021,DZ BRN,T_INT,Азия. Монголия,0.05,0.04,0.79,Rouble
1,01.06.2021,DZ BRN,T_INT,"Азия. Монголия, Моб.",2.68,2.12,0.79,Rouble
2,01.06.2021,DZ BRN,T_INT,Европа. Турция моб.Aycell,0.05,0.58,11.55,Rouble
3,01.06.2021,DZ BRN,T_INT,СНГ. Азербайджан моб.Azercell,16.98,290.42,17.10,Rouble
4,01.06.2021,DZ BRN,T_INT,СНГ. Казахстан,7.53,15.44,2.05,Rouble


In [180]:
df = raw.copy()
df.rename(cols_dict, axis=1, inplace=True)
if 'BRANCH' not in df.columns:
    df[['CARRIER', 'BRANCH']] = df['CARRIER'].str.extract(
        r'([A-Z]{1}\_[A-Z0-9]{3})\s([[A-Z0-9]{2}\s[A-Z0-9]*)', expand=True, 
    )        
df['CARRIER_TYPE'] = df['CARRIER'].str.extract(r'([A-Z]{1})\_[A-Z0-9]{3}', expand=True, )
df = df[(df['CARRIER_TYPE'].isin(['T','H'])) & (~df['CARRIER'].isin(['T_INT','H_RTK']))]
df['DIRECTION'] = 'Outgoing'
df['MINUTES'] = df['MINUTES'].apply(lambda x: pd.to_numeric(x, downcast='float'))
df['INSERT_DATE'] = datetime.today().strftime('%Y-%m-%d')
df.drop([col for col in ['#GLOBALCARRIER','MACROREGION'] if col in df.columns], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)
for date_format in ('%d.%m.%Y', '%d.%m.%y', '%d/%m/%Y'):
    try:
        df['CDR_DATE'] = df['CDR_DATE'].apply(lambda x: datetime.strptime(x, date_format).strftime('%Y-%m-%d'))
    except ValueError:
        continue
    else:
        df['CDR_MONTH'] = report_month = df['CDR_DATE'][0][0:-2] + '01'
        max_csv_date = max(df['CDR_DATE'])
        break
df.head()

,CDR_DATE,BRANCH,CARRIER,DESTINATION,MINUTES,COST,Tariff,CURRENCY,CARRIER_TYPE,DIRECTION,INSERT_DATE,CDR_MONTH
0,2021-06-01,DZ BRN,T_MTT,СНГ. Беларусь моб.MDC,0.75,16.92,22.556,Rouble,T,Outgoing,2021-06-09,2021-06-01
1,2021-06-01,DZ BRN,T_MTT,СНГ. Казахстан моб.KarTel,4.33,43.98,10.150,Rouble,T,Outgoing,2021-06-09,2021-06-01
2,2021-06-01,DZ BRN,T_MTT,СНГ. Казахстан моб.Telecom Service,2.03,27.76,13.654,Rouble,T,Outgoing,2021-06-09,2021-06-01
3,2021-06-01,DZ BRN,T_MTT,СНГ. Кыргызстан ~ Z2,0.78,8.48,10.827,Rouble,T,Outgoing,2021-06-09,2021-06-01
4,2021-06-01,DZ BRN,T_MTT,СНГ. Украина моб.Kyivstar,0.73,9.48,12.932,Rouble,T,Outgoing,2021-06-09,2021-06-01


In [225]:
query = '''
    sel * from uat_product.outgoing_mn_mapping
'''

with teradatasql.connect() as session:
    out_mn_mapping = pd.read_sql(query, session)
out_mn_mapping.columns = ['DESTINATION', 'DESTINATION_REGION_WO_TILDA', 'CODES', 'DESTINATION_WRONG', 'COUNTRY', 'AREA']
out_mn_mapping['CODES'] = out_mn_mapping['CODES'].apply(str)
out_mn_mapping['FIRST_CODE'] = out_mn_mapping['CODES'].apply(lambda x: x.split(',')[0].replace('810', ''))
df_out = pd.merge(df, out_mn_mapping, how='left', on='DESTINATION')
df_out.head()

KeyError: ('DESTINATION', 'FIRST_CODE')

In [193]:
def get_destination(code: str, dest_dict: dict):
    if len(code)>=2:
        for dest, codes_list in dest_dict.items():
            if code in codes_list:
                return dest
        else:
            code = code[:-1]
            return get_destination(code, dest_dict)
    else:
        return 'Other'
    

query = '''
    sel * from uat_product.outgoing_mn_dic
'''

with teradatasql.connect() as session:
    out_mn_dic = pd.read_sql(query, session)
out_mn_dic.columns = ['DIAL_CODES', 'NEW_DESTINATION']
dest_dict = {row['NEW_DESTINATION']:row['DIAL_CODES'].split(', ') for i, row in out_mn_dic.iterrows()}

df_out['DESTINATION'] = df_out['FIRST_CODE'].apply(lambda x: get_destination(str(x), dest_dict))
df_out.head()

,CDR_DATE,BRANCH,CARRIER,DESTINATION_x,MINUTES,COST,Tariff,CURRENCY,CARRIER_TYPE,DIRECTION,INSERT_DATE,CDR_MONTH,DESTINATION_REGION,DESTINATION_REGION_WO_TILDA,CODES,DESTINATION_y,COUNTRY,AREA,FIRST_CODE,DESTINATION
0,2021-06-01,DZ BRN,T_MTT,СНГ. Беларусь моб.MDC,0.75,16.92,22.556,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Беларусь моб.MDC,СНГ. Беларусь моб.MDC,"810375291, 810375293, 810375296, 810375299",Belarus Mobile MDC,Беларусь,СНГ,375291,Belarus Mobile MDC
1,2021-06-01,DZ BRN,T_MTT,СНГ. Казахстан моб.KarTel,4.33,43.98,10.150,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Казахстан моб.KarTel,СНГ. Казахстан моб.KarTel,"8107705, 8107771, 8107776, 8107777, 8705, 8771...",None,Казахстан,СНГ,7705,Kazakhstan Mobile KarTel
2,2021-06-01,DZ BRN,T_MTT,СНГ. Казахстан моб.Telecom Service,2.03,27.76,13.654,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Казахстан моб.Telecom Service,СНГ. Казахстан моб.Telecom Service,"8107707, 8107747, 8707, 8747",None,Казахстан,СНГ,7707,Kazakhstan Mobile Neo
3,2021-06-01,DZ BRN,T_MTT,СНГ. Кыргызстан ~ Z2,0.78,8.48,10.827,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Кыргызстан ~ Z2,СНГ. Кыргызстан - Z2,810996222,None,Кыргызстан,СНГ,996222,Kyrgyzstan
4,2021-06-01,DZ BRN,T_MTT,СНГ. Украина моб.Kyivstar,0.73,9.48,12.932,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Украина моб.Kyivstar,СНГ. Украина моб.Kyivstar,"81038068, 81038096, 81038097, 81038098",Ukraine Mobile Kyivstar,Украина,СНГ,38068,Ukraine Mobile Kyivstar


In [218]:
df_new.head()

,CARRIER,OPERATOR_NAME,CDR_DATE,COST,MINUTES,BRANCH,CARRIER_TYPE,ORIGINATSIJA,PRINADLEZHNOSTREGION,DIRECTION,INSERT_DATE
0,T_VPM,Болгария (мобил.) - BTC Mobile,16.05.2021,3.250000,0.216667,DZ BRT,T,Болгария,Европа,Incoming,2021-06-09
1,H_CHM,ОАЭ (мобил.) - Mobile others,15.05.2021,9710.054557,673.683333,T2 CFN,H,ОАЭ,Азия,Incoming,2021-06-09
2,T_VPM,Беларусь (мобил.) - BTN BeST,05.05.2021,44.500000,2.966667,RM PRM,T,Беларусь,СНГ,Incoming,2021-06-09
3,T_VPM,Швеция (мобил.) - Comviq,18.05.2021,1.500000,0.100000,T2 TUL,T,Швеция,Европа,Incoming,2021-06-09
4,T_VPM,Эстония (мобил.) - Tele2,15.05.2021,1072.500000,71.500000,T2 TAM,T,Эстония,Европа,Incoming,2021-06-09


In [293]:
df.head()

,#GLOBALCARRIER,CARRIER_SHORT_NAME,MACROREGION,OPERATOR_NAME,START_DATE,COST,DURATION
0,T_GTK,T_GTK T2 SPB,СНГ. Украина,Украина (мобил.)-UMC,03.06.2021,4.400000,.550000
1,M_MTS,M_MTS T2 NSK,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,05.06.2021,21.794000,21.366667
2,M_MGF,M_MGF T2 SPB,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,02.06.2021,.969000,.950000
3,T_INT,T_INT DZ BRT,ЕС. Великобритания,Великобритания (мобил.) - Vodafone,06.06.2021,.047500,.050000
4,M_MTS,M_MTS RM EKT,ЕС. Балтия. Эстония,Эстония,05.06.2021,.063333,.066667


In [298]:
df.head()

,#GLOBALCARRIER,CARRIER_SHORT_NAME,MACROREGION,OPERATOR_NAME,START_DATE,COST,DURATION
0,T_GTK,T_GTK T2 SPB,СНГ. Украина,Украина (мобил.)-UMC,03.06.2021,4.400000,.550000
1,M_MTS,M_MTS T2 NSK,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,05.06.2021,21.794000,21.366667
2,M_MGF,M_MGF T2 SPB,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,02.06.2021,.969000,.950000
3,T_INT,T_INT DZ BRT,ЕС. Великобритания,Великобритания (мобил.) - Vodafone,06.06.2021,.047500,.050000
4,M_MTS,M_MTS RM EKT,ЕС. Балтия. Эстония,Эстония,05.06.2021,.063333,.066667


In [301]:
df['NNN'] = np.NaN
df['NNNN'] = None
df.head()

,#GLOBALCARRIER,CARRIER_SHORT_NAME,MACROREGION,OPERATOR_NAME,START_DATE,COST,DURATION,NNN,NNNN
0,T_GTK,T_GTK T2 SPB,СНГ. Украина,Украина (мобил.)-UMC,03.06.2021,4.400000,.550000,NaN,None
1,M_MTS,M_MTS T2 NSK,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,05.06.2021,21.794000,21.366667,NaN,None
2,M_MGF,M_MGF T2 SPB,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,02.06.2021,.969000,.950000,NaN,None
3,T_INT,T_INT DZ BRT,ЕС. Великобритания,Великобритания (мобил.) - Vodafone,06.06.2021,.047500,.050000,NaN,None
4,M_MTS,M_MTS RM EKT,Эстония,Эстония,05.06.2021,.063333,.066667,NaN,None


In [304]:
df['NNN'] = df['NNN'].apply(lambda x: x if x else 'Unknown')
df.head()

,#GLOBALCARRIER,CARRIER_SHORT_NAME,MACROREGION,OPERATOR_NAME,START_DATE,COST,DURATION,NNN,NNNN
0,T_GTK,T_GTK T2 SPB,СНГ. Украина,Украина (мобил.)-UMC,03.06.2021,4.400000,.550000,NaN,None
1,M_MTS,M_MTS T2 NSK,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,05.06.2021,21.794000,21.366667,NaN,None
2,M_MGF,M_MGF T2 SPB,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,02.06.2021,.969000,.950000,NaN,None
3,T_INT,T_INT DZ BRT,ЕС. Великобритания,Великобритания (мобил.) - Vodafone,06.06.2021,.047500,.050000,NaN,None
4,M_MTS,M_MTS RM EKT,Эстония,Эстония,05.06.2021,.063333,.066667,NaN,None


In [305]:
df['NNNN'] = df['NNNN'].apply(lambda x: x if x else 'Unknown')
df.head()

,#GLOBALCARRIER,CARRIER_SHORT_NAME,MACROREGION,OPERATOR_NAME,START_DATE,COST,DURATION,NNN,NNNN
0,T_GTK,T_GTK T2 SPB,СНГ. Украина,Украина (мобил.)-UMC,03.06.2021,4.400000,.550000,NaN,Unknown
1,M_MTS,M_MTS T2 NSK,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,05.06.2021,21.794000,21.366667,NaN,Unknown
2,M_MGF,M_MGF T2 SPB,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,02.06.2021,.969000,.950000,NaN,Unknown
3,T_INT,T_INT DZ BRT,ЕС. Великобритания,Великобритания (мобил.) - Vodafone,06.06.2021,.047500,.050000,NaN,Unknown
4,M_MTS,M_MTS RM EKT,Эстония,Эстония,05.06.2021,.063333,.066667,NaN,Unknown


In [299]:
df['MACROREGION'] = df.apply(lambda x: x['OPERATOR_NAME'] if x['OPERATOR_NAME'] == 'Эстония' else x['MACROREGION'], axis=1)
df.head()

,#GLOBALCARRIER,CARRIER_SHORT_NAME,MACROREGION,OPERATOR_NAME,START_DATE,COST,DURATION
0,T_GTK,T_GTK T2 SPB,СНГ. Украина,Украина (мобил.)-UMC,03.06.2021,4.400000,.550000
1,M_MTS,M_MTS T2 NSK,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,05.06.2021,21.794000,21.366667
2,M_MGF,M_MGF T2 SPB,СНГ. Казахстан ДК Билайн,Казахстан (мобил.) - KaR-Tel,02.06.2021,.969000,.950000
3,T_INT,T_INT DZ BRT,ЕС. Великобритания,Великобритания (мобил.) - Vodafone,06.06.2021,.047500,.050000
4,M_MTS,M_MTS RM EKT,Эстония,Эстония,05.06.2021,.063333,.066667


In [220]:
df_new.columns

Index(['CARRIER', 'OPERATOR_NAME', 'CDR_DATE', 'COST', 'MINUTES', 'BRANCH',
       'CARRIER_TYPE', 'ORIGINATSIJA', 'PRINADLEZHNOSTREGION', 'DIRECTION',
       'INSERT_DATE'],
      dtype='object')

In [217]:
['CARRIER', 'OPERATOR_NAME', 'CDR_DATE', 'COST', 'MINUTES', 'BRANCH',
       'CARRIER_TYPE', 'ORIGINATSIJA', 'PRINADLEZHNOSTREGION', 'DIRECTION',
       'INSERT_DATE']

,CDR_DATE,BRANCH,CARRIER,DESTINATION_x,MINUTES,COST,Tariff,CURRENCY,CARRIER_TYPE,DIRECTION,INSERT_DATE,CDR_MONTH,DESTINATION_REGION,DESTINATION_REGION_WO_TILDA,CODES,DESTINATION_y,COUNTRY,AREA,FIRST_CODE,DESTINATION
0,2021-06-01,DZ BRN,T_MTT,СНГ. Беларусь моб.MDC,0.75,16.92,22.556,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Беларусь моб.MDC,СНГ. Беларусь моб.MDC,"810375291, 810375293, 810375296, 810375299",Belarus Mobile MDC,Беларусь,СНГ,375291,Belarus Mobile MDC
1,2021-06-01,DZ BRN,T_MTT,СНГ. Казахстан моб.KarTel,4.33,43.98,10.150,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Казахстан моб.KarTel,СНГ. Казахстан моб.KarTel,"8107705, 8107771, 8107776, 8107777, 8705, 8771...",None,Казахстан,СНГ,7705,Kazakhstan Mobile KarTel
2,2021-06-01,DZ BRN,T_MTT,СНГ. Казахстан моб.Telecom Service,2.03,27.76,13.654,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Казахстан моб.Telecom Service,СНГ. Казахстан моб.Telecom Service,"8107707, 8107747, 8707, 8747",None,Казахстан,СНГ,7707,Kazakhstan Mobile Neo
3,2021-06-01,DZ BRN,T_MTT,СНГ. Кыргызстан ~ Z2,0.78,8.48,10.827,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Кыргызстан ~ Z2,СНГ. Кыргызстан - Z2,810996222,None,Кыргызстан,СНГ,996222,Kyrgyzstan
4,2021-06-01,DZ BRN,T_MTT,СНГ. Украина моб.Kyivstar,0.73,9.48,12.932,Rouble,T,Outgoing,2021-06-09,2021-06-01,СНГ. Украина моб.Kyivstar,СНГ. Украина моб.Kyivstar,"81038068, 81038096, 81038097, 81038098",Ukraine Mobile Kyivstar,Украина,СНГ,38068,Ukraine Mobile Kyivstar


In [226]:
query = '''sel * from uat_product.outgoing_mn_mapping'''
with teradatasql.connect() as session:
    df = pd.read_sql(query, session)

In [227]:
df.columns = [
    'DESTINATION', 'DESTINATION_REGION_WO_TILDA', 'CODES', 'DESTINATION_WRONG', 'COUNTRY', 'AREA']
df.head()

,DESTINATION,DESTINATION_REGION_WO_TILDA,CODES,DESTINATION_WRONG,COUNTRY,AREA
0,"Европа. Испания, Моб. Telefonica ~ Z11","Европа. Испания, Моб. Telefonica - Z11",810346426,Spain Mobile Telefonica,Испания,Европа
1,Европа. Исландия моб.др.,Европа. Исландия моб.др.,"810354780, 810354781, 810354782, 810354783",None,Исландия,Европа
2,"Европа. Италия, Моб. Mobile others ~ Z6","Европа. Италия, Моб. Mobile others - Z6",8103974,None,Италия,Европа
3,Европа. Нидерланды моб.Vodafone роуминг,Европа. Нидерланды моб.Vodafone роуминг,81031654,None,Нидерланды,Европа
4,Америка. Панама моб. ~ Z2,Америка. Панама моб. - Z2,81050764,None,Панама,Америка


In [294]:
df['CODES'] = df['CODES'].apply(str)
df['FIRST_CODE'] = df['CODES'].apply(lambda x: x.split(',')[0].replace('810', ''))
df = df[['DESTINATION','FIRST_CODE']]
df.head()

KeyError: 'CODES'

In [ ]:
df['DESTINATION'] = df['FIRST_CODE'].apply(lambda x: get_destination(str(x), dest_dict))